https://github.com/comsa33/NLP-LSTM-emotion_classification/blob/main/N423a_Discussion2.ipynb

# 0) 모듈 실행

##기본 모듈

In [5]:
# Matplotlib 와 pandas 그래프에서 한글 사용 설정
import sys

## 노트북이 코랩에서 실행중인지 체크
if 'google.colab' in sys.modules:
  !echo 'debconf debconf/frontend select Noninterative' | debconf-set-selections
  ## 나눔 폰트 설치
  !sudo apt-get -qq -y install fonts-nanum
  import matplotlib.font_manager as fm
  font_files = fm.findSystemFonts(fontpaths=['/usr/share/fonts/truetype/nanum'])
  for fpath in font_files:
    fm.fontManager.addfont(fpath)

  # 나눔바른고딕 폰트 설정
  import matplotlib.pyplot as plt
  plt.rc('font', family='NanumBarunGothic')
  # 마이너스 기호 표시 오류 수정
  import matplotlib
  matplotlib.rcParams['axes.unicode_minus'] = False


debconf: unable to initialize frontend: Noninterative
debconf: (Can't locate Debconf/FrontEnd/Noninterative.pm in @INC (you may need to install the Debconf::FrontEnd::Noninterative module) (@INC contains: /etc/perl /usr/local/lib/x86_64-linux-gnu/perl/5.34.0 /usr/local/share/perl/5.34.0 /usr/lib/x86_64-linux-gnu/perl5/5.34 /usr/share/perl5 /usr/lib/x86_64-linux-gnu/perl-base /usr/lib/x86_64-linux-gnu/perl/5.34 /usr/share/perl/5.34 /usr/local/lib/site_perl) at (eval 17) line 2, <> line 1.)
debconf: falling back to frontend: Noninteractive
Selecting previously unselected package fonts-nanum.
(Reading database ... 121920 files and directories currently installed.)
Preparing to unpack .../fonts-nanum_20200506-1_all.deb ...
Unpacking fonts-nanum (20200506-1) ...
Setting up fonts-nanum (20200506-1) ...
Processing triggers for fontconfig (2.13.1-4.2ubuntu5) ...


In [6]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import f1_score
from sklearn.preprocessing import LabelEncoder
import random
import os

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [7]:
!pip install konlpy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.4/19.4 MB 53.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 488.6/488.6 kB 48.6 MB/s eta 0:00:00


In [8]:
import re
import urllib.request
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from collections import Counter

In [9]:
from konlpy.tag import Okt

okt = Okt()

# 1) 데이터 가져오기

## 감성대화 말뭉치
https://www.aihub.or.kr/aihubdata/data/view.do?currMenu=115&topMenu=100&aihubDataSe=realm&dataSetSn=86
- 일반인 1,500명 대상의 인터뷰 및 크라우드 소싱 수행
- 우울증 환자 대상 WOZ 대화 수집

=> 전처리 과정으로 "한사람만의 대화를 줄글로 변형 - 질문 제거"

### 1. 데이터 가져오기

In [150]:
train = pd.read_excel("/content/drive/MyDrive/0000/Class/졸프/Dataset/AIhub_감성대화/Training_221115_add/원천데이터/감성대화말뭉치(최종데이터)_Training.xlsx")
test = pd.read_excel("/content/drive/MyDrive/0000/Class/졸프/Dataset/AIhub_감성대화/Validation_221115_add/원천데이터/감성대화말뭉치(최종데이터)_Validation.xlsx")

In [151]:
train_df = train.copy()
test_df = test.copy()

### 2. 전처리

In [152]:
# "사람문장3" 결측값 대체
train_df.fillna('',inplace=True)
test_df.fillna('',inplace=True)

# 문장 병합
train_df['사람문장'] = train_df['사람문장1']+train_df['사람문장2']+train_df['사람문장3']
test_df['사람문장'] = test_df['사람문장1']+test_df['사람문장2']+test_df['사람문장3']

# "감정 분류"과 "문장"만 추출
train_df = train_df[['사람문장', '감정_대분류']]
test_df = test_df[['사람문장', '감정_대분류']]

# 컬럼명 변경
train_df.rename(columns={'사람문장': 'text',
                   '감정_대분류': 'sentiment'}, inplace=True)
test_df.rename(columns={'사람문장': 'text',
                   '감정_대분류': 'sentiment'}, inplace=True)

# label encoding
le = LabelEncoder()
le=le.fit(train_df['sentiment'])
train_df['target']=le.transform(train_df['sentiment'])

le=le.fit(test_df['sentiment'])
test_df['target']=le.transform(test_df['sentiment'])

# text, target
train_conv = train_df[['text','target']]
test_conv = test_df[['text','target']]

In [153]:
# 라벨 인코딩 결과 확인
sentiment = train_df[['sentiment', 'target']].drop_duplicates(subset=['sentiment']) # sentiment 컬럼에 대해 중복된 값을 갖고 있는 열 제거
df_unique = sentiment.sort_values(by='target', ascending=True)
df_unique = df_unique.reset_index(drop=True)
df_unique

,sentiment,target
0,기쁨,0
1,당황,1
2,분노,2
3,불안,3
4,상처,4
5,슬픔,5


# 2) 데이터 증강
데이터 불균형으로 인해 분류 전에 전처리하는 방식인 smote 택함

**SMOTE(= Synthetic Minority Over-sampling Technique)**

SMOTE는 합성을 기반으로 하는 방법으로, minority class의 샘플을 가져와 이들을 잇는 선에서 중간값을 택하여 만들어내는 방식

Train

In [154]:
# SMOTE : 오버샘플링
from imblearn.over_sampling import RandomOverSampler
text = train_conv.text.to_numpy().reshape(-1, 1)
target = train_conv.target.to_numpy().reshape(-1, 1)

oversample = RandomOverSampler()

X_oversample, y_oversample = oversample.fit_resample(text, target)
train_conv2 = pd.DataFrame({"text": X_oversample.reshape(-1), 'target': y_oversample.reshape(-1)})
train_conv2

,text,target
0,일은 왜 해도 해도 끝이 없을까? 화가 난다.그냥 내가 해결하는 게 나아. 남들한테...,2
1,이번 달에 또 급여가 깎였어! 물가는 오르는데 월급만 자꾸 깎이니까 너무 화가 나....,2
2,회사에 신입이 들어왔는데 말투가 거슬려. 그런 애를 매일 봐야 한다고 생각하니까 스...,2
3,직장에서 막내라는 이유로 나에게만 온갖 심부름을 시켜. 일도 많은 데 정말 분하고 ...,2
4,얼마 전 입사한 신입사원이 나를 무시하는 것 같아서 너무 화가 나.상사인 나에게 먼...,2
...,...,...
55915,부인에게 과거에 너무 못 해준 것 같아 많이 아쉬워.지금이라도 단둘이 여행이라도 가...,5
55916,나이가 들어서 그런지 와이프가 더 이상 운동을 못하게 해. 다치는 일이 잦아서 그런...,5
55917,요즘 치매가 부쩍 심해져서 금방 둔 물건도 잃어버리고 그래.나중엔 자식들도 잊어버리...,5
55918,정말 화가 나고 짜증 나.이제 슬슬 노후 준비해야 하는데 되는 일도 없고.아무리 열...,5


Test

In [155]:
# SMOTE : 오버샘플링
from imblearn.over_sampling import RandomOverSampler
text = test_conv.text.to_numpy().reshape(-1, 1)
target = test_conv.target.to_numpy().reshape(-1, 1)

oversample = RandomOverSampler()

X_oversample, y_oversample = oversample.fit_resample(text, target)
test_conv2 = pd.DataFrame({"text": X_oversample.reshape(-1), 'target': y_oversample.reshape(-1)})
test_conv2

,text,target
0,이번 프로젝트에서 발표를 하는데 내가 실수하는 바람에 우리 팀이 감점을 받았어. 너...,3
1,회사에서 중요한 프로젝트를 혼자 하게 됐는데 솔직히 두렵고 무서워.나에게 너무 크게...,3
2,상사가 너무 무섭게 생겨서 친해지는 게 너무 두려워.무섭게 생겼는데도 업무를 보려면...,3
3,이번에 힘들게 들어간 첫 직장이거든. 첫 직장이라서 그런지 너무 긴장된다.첫 직장이...,3
4,직장에서 동료들이랑 관계가 안 좋아질까 봐 걱정돼.내가 낯가림이 심해서 친해질 수 ...,3
...,...,...
7537,아내가 오래된 지병으로 결국 세상을 떠났어.아내가 너무 보고 싶어. 아내에게 해줄 ...,5
7538,내가 사랑하는 남편이 죽었어. 평생을 같이 살았는데 먼저 가버려서 너무 슬퍼.괜찮지...,5
7539,그동안 와이프한테 내가 너무 무심했던 것 같아서 후회가 돼.결혼기념일도 제대로 챙긴...,5
7540,우리 엄마는 새벽까지 잠 안 자고 공부하는 모습을 지켜봐. 날 못 믿는 거 같아서 ...,5


# 3) 데이터 전처리
모델 적합을 위해 데이터가 적절히 전처리되어야 하며 이때 전처리는 토큰화, 정수 인코딩, 패딩을 의미함

- 불용어 적용 및 토큰화
- 데이터 분할
- 단어 집합 생성
- 정수 인코딩
- 패딩

In [156]:
features = train_conv2[['text']]
targets = train_conv2[['target']]

features.shape, targets.shape

((55920, 1), (55920, 1))

불용어 적용 및 토큰화

In [22]:
# 한국어 불용어 사전 적용 https://bab2min.tistory.com/544
'''with open('/content/한국어불용어100.txt') as f:
    lines = f.readlines()
stopwords = []
for stopword in lines:
    stopwords.append(stopword.split('\t')[0])
print(stopwords)
'''
stopwords = ['이', '있', '하', '것', '들', '그', '되', '수', '이', '보', '않', '없', '나', '사람',
              '주', '아니', '등', '같', '우리', '때', '년', '가', '한', '지', '대하', '오', '말', '일',
              '그렇', '위하', '때문', '그것', '두', '말하', '알', '그러나', '받', '못하', '일', '그런', '또',
              '문제', '더', '사회', '많', '그리고', '좋', '크', '따르', '중', '나오', '가지', '씨', '시키',
              '만들', '지금', '생각하', '그러', '속', '하나', '집', '살', '모르', '적', '월', '데', '자신',
              '안', '어떤', '내', '내', '경우', '명', '생각', '시간', '그녀', '다시', '이런', '앞', '보이',
              '번', '나', '다른', '어떻', '여자', '개', '전', '들', '사실', '이렇', '점', '싶', '말', '정도',
              '좀', '원', '잘', '통하', '소리', '놓']


In [158]:
features['text'] = features['text'].str.replace("[^ ㄱ-ㅣ가-힣a-zA-Z0-9+]", " ")
features['text'] = features['text'].apply(lambda x: ' '.join([w for w in x.split() if w not in stopwords]))

<ipython-input-158-970d5fe97f06>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  features['text'] = features['text'].str.replace("[^ ㄱ-ㅣ가-힣a-zA-Z0-9+]", " ")
<ipython-input-158-970d5fe97f06>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  features['text'] = features['text'].apply(lambda x: ' '.join([w for w in x.split() if w not in stopwords]))


데이터 분할

In [160]:
!pip install category_encoders

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.9/81.9 kB 1.1 MB/s eta 0:00:00


In [162]:
from sklearn.model_selection import train_test_split
from category_encoders import OrdinalEncoder

enc = OrdinalEncoder()
target = enc.fit_transform(targets['target'])
X_train, X_test, y_train, y_test = train_test_split(features['text'], target,
                                                    test_size=0.15, stratify=target,
                                                    random_state=33)

X_train.shape, X_test.shape

((47532,), (8388,))

In [163]:
X_train.head()

22607    과가 없어지고 과와 통합된대. 내가 다니던 과가 없어져서 당황스러워.추억이 없어지는...
40179    내게 딸이 둘 있어. 첫째는 시집가서 아들만 둘이고 둘째 딸은 공부만 하고 시집갈 ...
3634     다이어트해서 십 킬로그램이나 체중을 줄였어.회사도 가고 아이도 봐야 하고 집안일까지...
23436    남편이 이혼할 위자료를 주지 않으려고 변호사를 고용했어.어떻게 그럴 수가 있지? 너...
54144    손자들은 내가 보고 싶어서 방학되면 무조건 병문안을 올 거야. 정말 기뻐.남들은 손...
Name: text, dtype: object

In [176]:
import tensorflow as tf
from tensorflow.keras.layers import Dense, BatchNormalization, LSTM, Embedding, SimpleRNN, RNN, LSTMCell
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing import sequence

y_train = tf.one_hot(y_train['target'], 6)
y_test = tf.one_hot(y_test['target'], 6)
y_train

<tf.Tensor: shape=(47532, 6), dtype=float32, numpy=
array([[0., 0., 0., 0., 1., 0.],
       [0., 0., 1., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0.],
       ...,
       [0., 0., 0., 0., 0., 1.],
       [0., 0., 0., 1., 0., 0.],
       [0., 0., 0., 1., 0., 0.]], dtype=float32)>

In [166]:
token = Tokenizer(num_words=6000)

In [167]:
token.fit_on_texts(X_train)
len(token.word_index)

102355

In [168]:
X_train_enc = token.texts_to_sequences(X_train)

In [169]:
max(len(x) for x in X_train_enc)

67

In [170]:
np.mean([len(x) for x in X_train_enc])

18.79338130101826

In [171]:
X_train = sequence.pad_sequences(X_train_enc, maxlen=15, padding='post')
X_train

array([[2774,    2, 1395, ...,    0,    0,    0],
       [  39,   10,  178, ...,  261,  198,  209],
       [   5,   42,  837, ...,   28,  308,  187],
       ...,
       [ 794, 2906,  689, ...,  465, 2915,  656],
       [3090,  166,   24, ...,    0,    0,    0],
       [2377, 2495,  214, ...,  278, 3520,   14]], dtype=int32)

In [172]:
vocab_size = len(token.word_index) + 1
vocab_size

102356

In [141]:
!pip install keras_tuner

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.1/129.1 kB 1.4 MB/s eta 0:00:00


In [177]:
import IPython
import keras_tuner as kt
from tensorflow import keras

tf.random.set_seed(33)

def model_builder(hp):

    output_size = 6
    hp_units = hp.Int('units', min_value = 40, max_value = 96, step = 8)
    hp_units1 = hp.Int('units1', min_value = 40, max_value = 96, step = 8)
    hp_units2 = hp.Int('units2', min_value = 40, max_value = 96, step = 8)
    hp_learning_rate = hp.Choice('learning_rate', values = [1e-2])

    lstm_layer = keras.layers.LSTM(hp_units,
                                   activation='tanh',
                                   recurrent_activation='sigmoid')

    model = keras.models.Sequential(
        [
            keras.layers.Input(shape=(15,)),
            keras.layers.Embedding(vocab_size, 64, input_length=15),
            lstm_layer,
            keras.layers.BatchNormalization(),
            keras.layers.Dense(hp_units1, activation='relu'),
            keras.layers.Dense(hp_units2, activation='relu'),
            keras.layers.Dense(output_size, activation='softmax'),
        ]
    )

    model.compile(loss='categorical_crossentropy',
                  optimizer=tf.keras.optimizers.Adam(learning_rate=hp_learning_rate),
                  metrics=['accuracy'])
    return model

tuner = kt.Hyperband(model_builder,
                     objective = 'val_accuracy',
                     max_epochs = 100,
                     factor = 3,
                     directory = 'my_dir',
                     project_name = 'kt9_LSTM_result')

class ClearTrainingOutput(tf.keras.callbacks.Callback):
    def on_train_end(*args, **kwargs):
        IPython.display.clear_output(wait = True)

early_stop = tf.keras.callbacks.EarlyStopping(monitor='val_accuracy',
                                           min_delta=0,
                                           patience=5,
                                           verbose=1)

checkpoint_filepath = "best_a.hdf5"
save_best = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    monitor='val_accuracy',
    verbose=1,
    save_best_only=True,
    save_weights_only=True,
    mode='auto',
    save_freq='epoch',
    options=None)

tuner.search(X_train, y_train,
             epochs=100,
             batch_size=800,
             callbacks=[ClearTrainingOutput(), early_stop, save_best],
             validation_split=0.15)

Trial 39 Complete [00h 00m 27s]
val_accuracy: 0.5422160029411316

Best val_accuracy So Far: 0.5586255192756653
Total elapsed time: 00h 20m 51s

Search: Running Trial #40

Value             |Best Value So Far |Hyperparameter
96                |40                |units
48                |64                |units1
40                |40                |units2
0.01              |0.01              |learning_rate
2                 |2                 |tuner/epochs
0                 |0                 |tuner/initial_epoch
4                 |4                 |tuner/bracket
0                 |0                 |tuner/round

Epoch 1/2
51/51 [==============================] - ETA: 0s - loss: 1.3806 - accuracy: 0.4568

KeyboardInterrupt: 

In [178]:
best_hps = tuner.get_best_hyperparameters(num_trials = 1)[0]
best_hps

In [179]:
model = tuner.hypermodel.build(best_hps)

model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 15, 64)            6550784   
                                                                 
 lstm_1 (LSTM)               (None, 40)                16800     
                                                                 
 batch_normalization_1 (Bat  (None, 40)                160       
 chNormalization)                                                
                                                                 
 dense_3 (Dense)             (None, 64)                2624      
                                                                 
 dense_4 (Dense)             (None, 40)                2600      
                                                                 
 dense_5 (Dense)             (None, 6)                 246       
                                                      

In [180]:
model.fit(X_train, y_train,
          epochs=100,
          batch_size=600,
          callbacks=[early_stop, save_best],
          validation_split=0.15)

Epoch 1/100
68/68 [==============================] - ETA: 0s - loss: 1.3566 - accuracy: 0.4655
Epoch 1: val_accuracy improved from -inf to 0.53661, saving model to best_a.hdf5
68/68 [==============================] - 13s 128ms/step - loss: 1.3566 - accuracy: 0.4655 - val_loss: 1.5251 - val_accuracy: 0.5366
Epoch 2/100
68/68 [==============================] - ETA: 0s - loss: 1.0354 - accuracy: 0.6220
Epoch 2: val_accuracy did not improve from 0.53661
68/68 [==============================] - 7s 96ms/step - loss: 1.0354 - accuracy: 0.6220 - val_loss: 1.4109 - val_accuracy: 0.5321
Epoch 3/100
68/68 [==============================] - ETA: 0s - loss: 0.9233 - accuracy: 0.6653
Epoch 3: val_accuracy improved from 0.53661 to 0.55863, saving model to best_a.hdf5
68/68 [==============================] - 4s 65ms/step - loss: 0.9233 - accuracy: 0.6653 - val_loss: 1.2689 - val_accuracy: 0.5586
Epoch 4/100
68/68 [==============================] - ETA: 0s - loss: 0.8199 - accuracy: 0.7033
Epoch 4: val

In [182]:
model.load_weights(checkpoint_filepath)
test_loss, test_acc = model.evaluate(X_test, y_test, verbose=1)

UnimplementedError: Graph execution error:

Detected at node sequential_1/Cast defined at (most recent call last):
  File "/usr/lib/python3.10/runpy.py", line 196, in _run_module_as_main

  File "/usr/lib/python3.10/runpy.py", line 86, in _run_code

  File "/usr/local/lib/python3.10/dist-packages/colab_kernel_launcher.py", line 37, in <module>

  File "/usr/local/lib/python3.10/dist-packages/traitlets/config/application.py", line 992, in launch_instance

  File "/usr/local/lib/python3.10/dist-packages/ipykernel/kernelapp.py", line 619, in start

  File "/usr/local/lib/python3.10/dist-packages/tornado/platform/asyncio.py", line 195, in start

  File "/usr/lib/python3.10/asyncio/base_events.py", line 603, in run_forever

  File "/usr/lib/python3.10/asyncio/base_events.py", line 1909, in _run_once

  File "/usr/lib/python3.10/asyncio/events.py", line 80, in _run

  File "/usr/local/lib/python3.10/dist-packages/tornado/ioloop.py", line 685, in <lambda>

  File "/usr/local/lib/python3.10/di

In [183]:
from sklearn.metrics import classification_report

y_pred = np.argmax(model.predict(X_test), axis=1)
print(classification_report(y_test, y_pred))

UnimplementedError: Graph execution error:

Detected at node sequential_1/Cast defined at (most recent call last):
  File "/usr/lib/python3.10/runpy.py", line 196, in _run_module_as_main

  File "/usr/lib/python3.10/runpy.py", line 86, in _run_code

  File "/usr/local/lib/python3.10/dist-packages/colab_kernel_launcher.py", line 37, in <module>

  File "/usr/local/lib/python3.10/dist-packages/traitlets/config/application.py", line 992, in launch_instance

  File "/usr/local/lib/python3.10/dist-packages/ipykernel/kernelapp.py", line 619, in start

  File "/usr/local/lib/python3.10/dist-packages/tornado/platform/asyncio.py", line 195, in start

  File "/usr/lib/python3.10/asyncio/base_events.py", line 603, in run_forever

  File "/usr/lib/python3.10/asyncio/base_events.py", line 1909, in _run_once

  File "/usr/lib/python3.10/asyncio/events.py", line 80, in _run

  File "/usr/local/lib/python3.10/dist-packages/tornado/ioloop.py", line 685, in <lambda>

  File "/usr/local/lib/python3.10/di